In [2]:
import numpy as np
import pandas as pd
import os
import sys

In [3]:
res_folder = "./disagreement_matrix/engineering_version/auc_results/"

In [12]:
auc_results = dict()
for root, path, files in os.walk(res_folder):
    for file in files:
        if file.endswith("xlsx"):
            auc_results[file[:-5]] = pd.read_excel(root+file, index_col=0)

In [36]:
# key as comb, value is fname
surpass_record = dict()

for fname in auc_results.keys():
    df_ = auc_results[fname]
    best_base_auc = df_.auc[df_.combination=="best_base_auc"].values[0]
    good_records = df_.combination[df_.auc>best_base_auc]
    if len(good_records)>0:
        for m in good_records.values:
            if m in surpass_record.keys():
                surpass_record[m].add(fname)
            else:
                surpass_record[m] = set([fname])

In [39]:
res_list = []
for key, value in surpass_record.items():
    res_list.append((key, len(value)))

        base_best (auc) |   merge_best
file1
file2

In [46]:
all_files_record = {"best_base_auc":[], "ensemble_best_auc":[]}
fnames = []
for fname in auc_results.keys():
    df_ = auc_results[fname]
    best_base_auc = df_.auc[df_.combination=="best_base_auc"].values[0]
    merge_best = np.max(df_.auc[df_.combination!="best_base_auc"].values)
    all_files_record['best_base_auc'].append(best_base_auc)
    all_files_record['ensemble_best_auc'].append(merge_best)
    fnames.append(fname)
    
all_files_record['DataSets'] = fnames

In [48]:
df_res = pd.DataFrame.from_dict(all_files_record)

In [57]:
pd.DataFrame(sorted(res_list,key=lambda x: 
                    x[1],reverse=True),columns=['ensemble_methods',
                    'num_surpass']).to_excel("./LOF_base_detectors_ensemble.xlsx")

In [50]:
df_res['best_base_auc'] = np.round(df_res['best_base_auc'],3)
df_res['ensemble_best_auc'] = np.round(df_res['ensemble_best_auc'],3)

In [52]:
df_res['diff'] = df_res['ensemble_best_auc'] - df_res['best_base_auc']

In [58]:
q = df_res.loc[:,['ensemble_best_auc','best_base_auc','diff','DataSets']]

In [60]:
q.to_excel("./LOF_base_overall_perform.xlsx")